## Title: bokk rating predicition 
### Date stated:13/01/2023
### Author: Pierre Mulliez 
### Kaggle link: https://www.kaggle.com/competitions/goodreads-books-reviews-290312/data

In [168]:
# import packages 
import pandas as pd

In [169]:
#load data
books = pd.read_csv('./data/goodreads_train.csv')
books_raw = books.copy()
books_to_rate = pd.read_csv('./data/goodreads_test.csv')

In [170]:
print(books.describe()) 
print('TO RATE:')
print(books_to_rate.describe())

            book_id         rating        n_votes     n_comments
count  9.000000e+05  900000.000000  900000.000000  900000.000000
mean   1.344145e+07       3.689639       3.294381       1.016656
std    9.357863e+06       1.252583      17.873553       5.963821
min    1.000000e+00       0.000000      -3.000000      -1.000000
25%    6.340471e+06       3.000000       0.000000       0.000000
50%    1.344203e+07       4.000000       0.000000       0.000000
75%    2.057897e+07       5.000000       2.000000       0.000000
max    3.632868e+07       5.000000    3222.000000    1335.000000
TO RATE:
            book_id        n_votes     n_comments
count  4.780330e+05  478033.000000  478033.000000
mean   1.320363e+07       2.721015       0.797947
std    9.071462e+06      15.212006       4.096425
min    1.000000e+00      -3.000000      -2.000000
25%    6.489636e+06       0.000000       0.000000
50%    1.320676e+07       0.000000       0.000000
75%    1.877525e+07       1.000000       0.000000
max   

In [171]:
#no null values
for col in books.columns:
    print(col)
    print(books.loc[books[col].isnull() == True,:].count()[0])

user_id
0
book_id
0
review_id
0
rating
0
review_text
0
date_added
0
date_updated
0
read_at
91766
started_at
274297
n_votes
0
n_comments
0


In [172]:
books['review_text'].tail(50)

899950               I need therapy after reading this book
899951    THIS SHOULD BE 4 STARS NOT 5 BUT IT MADE ME FE...
899952    THE WHOLE LAST HALF OF THIS BOOK IS LIKE [TAKE...
899953    Me the first half of this book: this is so fuc...
899954    I love slice-of-life stories where pretty much...
899955    "Did I ever tell you why I love Kate Bishop? I...
899956    I feel like counting these comics as books I'v...
899957    I MANAGED TO FINISH A 500+ PAGE BOOK FOR THE F...
899958    Tearjerker alert! \n Oh gosh. Everything still...
899959    Full review here: http://bookaholicfairies.blo...
899960    I'm not very familiar with the Mayan culture b...
899961    The Characters: \n Ben is such a charming char...
899962    4.5 stars \n The Characters: \n I have always ...
899963    The story picks up where it left off in book o...
899964    4 Stars! \n Lincoln is a 28-year old employee ...
899965    When The Scribe went live in October 2013, I w...
899966    REVIEW CONTAIN SPOILERS \n Hon

In [173]:
books['n_comments'].head(40)

0      1
1      0
2      0
3      1
4      1
5      5
6      3
7      5
8      6
9      6
10     1
11     4
12     1
13     0
14     2
15     5
16     0
17     2
18     1
19     5
20     1
21     0
22     0
23     1
24     4
25     1
26     1
27     3
28     1
29     3
30    10
31    25
32     3
33     2
34     0
35     1
36     2
37     0
38     3
39     9
Name: n_comments, dtype: int64

In [174]:
#Null values
def isNaN(string):
    return string != string

#week = {'Mon':1,'Tue':2,'Wed':3,'Thu':4,'Fri':5,'Sat':6,'Sun':7}
month = {'Jan': '01' ,'Feb':'02', 'Mar':'03','Apr':'04', 'May':'05', 'Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}
 
    
for num in range(0, 900000):
    if (isNaN(books.loc[num,'started_at']) != True):
        books.loc[num,'date_started'] =  datetime.strptime((month.get(books.loc[2,'started_at'][4:7]) + '/' + str(books.loc[2,'started_at'][-4:])), '%m/%Y')
    elif (isNaN(books.loc[num,'read_at']) != True) : 
        books.loc[num,'date_started'] = datetime.strptime((month.get(books.loc[2,'read_at'][4:7]) + '/' + str(books.loc[2,'read_at'][-4:])), '%m/%Y')
    else: 
        books.loc[num,'date_started'] = datetime.strptime((month.get(books.loc[2,'date_added'][4:7]) + '/' + str(books.loc[2,'date_added'][-4:])), '%m/%Y')

    if (isNaN(books.loc[num,'read_at']) != True):
        books.loc[num,'date_read'] = datetime.strptime((month.get(books.loc[2,'read_at'][4:7]) + '/' + str(books.loc[2,'read_at'][-4:])), '%m/%Y')
    elif (isNaN(books.loc[num,'read_at']) != True) : 
        books.loc[num,'date_read'] = datetime.strptime((month.get(books.loc[2,'started_at'][4:7]) + '/' + str(books.loc[2,'started_at'][-4:])), '%m/%Y')

In [175]:
#work on month format
print(books.loc[2,'read_at'][4:7])
month.get(books.loc[2,'read_at'][4:7])
from datetime import datetime
print(datetime.strptime((month.get(books.loc[2,'read_at'][4:7]) + '/' + str(books.loc[2,'read_at'][-4:])),   '%m/%Y'))

Mar
2017-03-01 00:00:00


In [177]:
#feature enrichment 

positive = ['fun','best', 'enjoy','enjoyed','appreciate','appreciated' \
            , 'approve','approved', 'exellent','recommend','recommended','good' \
           'great', 'loved','love', '5 stars','amazing']

negative = ['not fun', 'not enjoy','not appreciate', 'not approve', 'not recommend' \
            ,'bad','too long','too short','worst','unreadable','hate']

def positivity(sentence):
    feeling = 0
    for word in sentence.split(' '): 
        if word in positive:
            feeling += 1
        elif word in negative:
            feeling -= 1
    return feeling
            
books['emotion'] = [positivity(books.loc[num,'review_text']) for num in range(0, len(books['review_text']))]

#active commentor to their number of votes
books['proportional_comment'] = books['n_votes'] / \
                                [1 if books.loc[num,'n_comments'] == 0 else books.loc[num,'n_comments'] for num in range(0, len(books['n_comments']))]

#time elaspsed between book started and finished
books['time_started_read'] =  books['date_started'] - books['date_read']


In [178]:
print(min(books['time_started_read']))

print(max(books['time_started_read']))
books.head(30)

0 days 00:00:00
0 days 00:00:00


,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,date_started,date_read,emotion,proportional_comment,time_started_read
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1,2017-03-01,2017-03-01,2,28.000000,0 days
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0,2016-11-01,NaT,0,1.000000,NaT
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0,2017-03-01,2017-03-01,1,22.000000,0 days
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1,2016-11-01,NaT,0,5.000000,NaT
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1,2017-03-01,2017-03-01,6,9.000000,0 days
5,8842281e1d1347389f2ab93d60773d4d,19398490,ea4a220b10e6b5c796dae0e3b970aff1,4,A beautiful story. It is rare to encounter a b...,Sun Jan 03 21:20:46 -0800 2016,Tue Sep 20 23:30:15 -0700 2016,Tue Sep 13 11:51:51 -0700 2016,Sat Aug 20 07:03:03 -0700 2016,35,5,2017-03-01,2017-03-01,1,7.000000,0 days
6,8842281e1d1347389f2ab93d60773d4d,22318578,5fe9882bfe4b0520a322820c4c55747d,5,5 stars for giving me a better framework for h...,Sun Jun 07 12:50:13 -0700 2015,Wed Mar 22 11:36:58 -0700 2017,Sun Aug 09 00:00:00 -0700 2015,Sun Jun 07 00:00:00 -0700 2015,24,3,2017-03-01,2017-03-01,0,8.000000,0 days
7,8842281e1d1347389f2ab93d60773d4d,24189224,dbc01e2438df7a87ee3dc16ee23a53e5,0,Numerous people in publishing have told me thi...,Fri May 29 17:48:57 -0700 2015,Fri May 29 17:49:40 -0700 2015,NaN,NaN,11,5,2016-11-01,NaT,0,2.200000,NaT
8,8842281e1d1347389f2ab93d60773d4d,22551730,754710070f38e1c59a45f6a85473b836,4,Another hard to put down nonfiction book from ...,Thu May 07 13:59:32 -0700 2015,Wed Dec 14 12:30:43 -0800 2016,Sat Jul 11 00:00:00 -0700 2015,Wed Jun 24 00:00:00 -0700 2015,20,6,2017-03-01,2017-03-01,1,3.333333,0 days
9,8842281e1d1347389f2ab93d60773d4d,22816087,d11954e6e9e9ddf52aed41d83ce43fa6,5,I love Stephenson - and this was another hit -...,Wed Apr 29 10:56:44 -0700 2015,Wed Dec 14 12:30:43 -0800 2016,Wed Nov 04 00:00:00 -0800 2015,Sun Sep 27 00:00:00 -0700 2015,40,6,2017-03-01,2017-03-01,1,6.666667,0 days


In [179]:
books.loc[:,('book_id', 'n_votes','n_comments','proportional_comment')]

,book_id,n_votes,n_comments,proportional_comment
0,18245960,28,1,28.0
1,16981,1,0,1.0
2,28684704,22,0,22.0
3,27161156,5,1,5.0
4,25884323,9,1,9.0
...,...,...,...,...
899995,11832081,0,0,0.0
899996,16095092,1,1,1.0
899997,8430896,1,0,1.0
899998,12275680,0,0,0.0


In [180]:
books.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,date_started,date_read,emotion,proportional_comment,time_started_read
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1,2017-03-01,2017-03-01,2,28.0,0 days
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0,2016-11-01,NaT,0,1.0,NaT
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0,2017-03-01,2017-03-01,1,22.0,0 days
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1,2016-11-01,NaT,0,5.0,NaT
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1,2017-03-01,2017-03-01,6,9.0,0 days


In [181]:
books_raw.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1
